In [ ]:
#upload glove.6B.300d.txt
import os
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2021-03-18 17:59:50--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-03-18 17:59:51--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-03-18 17:59:51--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [5]:
import urllib.request
url = 'http://nlp.stanford.edu/data/glove.6B.zip'
filename = 'glove6B.6B.zip'
urllib.request.urlretrieve(url, filename)

('glove6B.6B.zip', <http.client.HTTPMessage at 0x11d163417f0>)

In [ ]:
#upload small.csv file
from google.colab import files
files.upload()

{}

In [43]:
import pandas as pd

test = pd.read_csv("small.csv")
test.info()
test

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   reviewText  99946 non-null   object
 1   overall     100000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.5+ MB


,reviewText,overall
0,They look good and stick good! I just don't li...,4
1,These stickers work like the review says they ...,5
2,These are awesome and make my phone look so st...,5
3,Item arrived in great time and was in perfect ...,4
4,"awesome! stays on, and looks great. can be use...",5
...,...,...
99995,I ordered the Dr. Green/Solid White version fr...,4
99996,I bought this cover for my husband's phone and...,5
99997,"The case has really nice colors, very vibrant....",5
99998,The replacement is on its way. I recommend thi...,5


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#the following working directory should contain small.csv and glove.6B.300d.txt 
os.chdir('/content/drive/My Drive/Teaching/teaching at Lehigh/2021_sp_nlp/Project 1/data/')


## Text Preprocessing and Dataset Construction

### Define the WordIndexer class to
*   hold the mapping from words to their indices and the indices to words.
*   map from a list of sentences to a list of integers so that words are mapped to their indices, in the same order as the original words (except some words replaced).

### Inherit from the `torch.utils.data.Dataset` class and create the AmazonReviewGloveDataset class to


*   load the Amazon reviews in the csv format. Tokenize the review texts into sentences (a review can contain more than one sentence).
*   use the WordIndexer class to obtain the indices of the words in the sentences.
*   compute the X (word co-occurrence) matrix as the Glove paper indicates.

We provide the function to read the pretrained word vectors from text files.

In [62]:
import re
from collections import Counter
import pickle

from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import nltk
import itertools
nltk.download('punkt')
nltk.download('stopwords')

class WordIndexer:
    """Transform a dataset of text to a list of index of words."""

    def __init__(self, min_word_occurrences=10, oov_word="OOV"):
        """ min_word_occurrences: integer, the minimum frequency of the word to keep.
            oov_word: string, a special string for out-of-vocabulary words.
        """
        self.oov_word = oov_word
        self.min_word_occurrences = min_word_occurrences
        # word to integer index mapping
        self.word_to_index = {oov_word: 0}
        # the inverse of the above mapping
        self.index_to_word = [oov_word]
        # this is for storing the word frequencies for removing infrequent words
        self.word_occurrences = {}
        # regular expression for retaining meaningful English words
        self.re_words = re.compile(r"\b[a-zA-Z]{2,}\b")

    def get_word_index(self, word, add_new_word = True):
        """ Find the index of a word.
                
            word: string, the query word.
            add_new_word: if true and the word has no entry, assign a new integer index to word.
                            if false, return the index of the oov_word
        """
        ### Your codes go here (10 points) ###  ###COMPLETE###
        if word not in self.index_to_word:
          if add_new_word == True: 
            self.index_to_word.append(word)
            self.word_to_index[word] = self.index_to_word.index(word)
          else:
            return -1
        else:
          return self.index_to_word.index(word)
      
    @property
    def n_words(self):
        """ return: the vocabulary size
        """
        return len(self.word_to_index)

    def fit_transform(self, texts):
        """ texts: list of sentences, each of which is a string
            
            Split each sentence into a list of words.
            Then filter out the infrequent words.
            Other text preprocessing, such as
                lower-casing,
                stop-word removal, and
                advance word tokenization
                are possible here.
            Lastly setup the word-to-index and index-to-word dictionaries.
            
            return: a list of lists of indices of words in each sentence.
                    For example: [[1,2,3], [4,5,6]] where,
                        [1,2,3] are the indices of words in the first sentence
                        [4,5,6] are the indices of words in the second sentence
                    
        """
        #STEP 1 fit_transform 
        #obtain stop words in root folder
        ### Your codes go here (10 points) ###
        ####COMPLETE#####CAN ADD MORE THINGS TO MAKE WORDS BETTER####

        f_stopwords = open(r"C:\Users\chase\AppData\roaming\nltk_data\corpora\stopwords\english", 'r', encoding="utf8")

        #take words from file and sanitize
        stopwords_english = []
        for line in f_stopwords:
          clean_line = line.replace("\n", "")
          stopwords_english.append(clean_line)
                
        words = [] #list of single words to help with counter class (counter doesnt work on list of lists)
        list_sentences_to_words = [] #future corupus

        print("Tokenizing sentences into words...")
        for sentence in texts:
          sentence_to_word = nltk.tokenize.word_tokenize(sentence) #convert sentences to list of words
          filtered_sentence = []
          for word in sentence_to_word: 
            if word.lower() not in stopwords_english:  #filter sentence of stopwords and convert to lowercase
              filtered_sentence.append(word.lower())
              words.append(word.lower())  
          list_sentences_to_words.append(filtered_sentence)
        print("Done implementing tokenization and taking out stopwords.")
      
          # Step 2 fit_transform: Build a dictionary using the Counter class
        # keep the unique words and their counts
        # filter out the infrequent ones using the threshold self.min_word_occurrences.
        # the results is a vocabulary in self.word_to_index and self.index_to_word.
        ### Your codes go here (10 points)
         ######COMPLETE#####

        print("Building a dictionary with occurrences of words...")
        self.word_occurrences = dict(Counter(words)) #get list of occurrences in words list (list of words from sentences) using counter class

        print("Removing words that don't meet the minimum word occurrences...")
        for word in list(self.word_occurrences):
            if self.word_occurrences.get(word) < self.min_word_occurrences:
              self.word_occurrences.pop(word)

        self.index_to_word.extend(list(self.word_occurrences))

        for word in self.index_to_word:
            self.word_to_index[word] = self.index_to_word.index(word) #inverse of index_to_word
        print("Vocabulary in word_to_index and index_to_word.")


        # save the word and their counts to a file.

        with open('./train_word_counts.txt', 'w') as out_f:
            a = sorted([(word, count) for word, count in self.word_occurrences.items()],
                   key = lambda x:x[1], reverse=True)
            for word, count in a:
                out_f.write('{}:{}\n'.format(word, count))

        # Step 3 fit_transform: build and return the corpus in index representation
        # using the vocabulary built in the last step.
        # Be careful about words that are not in the vocabulary.
        ### Your codes go here (10 points) ###
        ###COMPLETE####

        corpus = []
        for sentence in list_sentences_to_words:
            corpus_sentence = []
            for word in sentence:
              res = self.word_to_index.get(word)
              if res is None:
                corpus_sentence.append(0)
              else:
                corpus_sentence.append(res)
            corpus.append(corpus_sentence)

        return corpus

    
class AmazonReviewGloveDataset(Dataset):
    def __init__(self, path, right_window = 4, min_word_occurrences = 10):        
        """ Load the reviews from a csv file. One row is one review.
                
            path: path to the csv file containing the reviews and their ratings
            right_window: integer, how large the window is to get context words.
            min_word_occurrences: integer, the minimum frequency of the word to keep.

            No return value
        """
        self.right_window = right_window

        # Step 1: tokenize the first field of each row in the csv file into sentences
        #         (e.g. using nltk.tokenize.sent_tokenize).
        #           Use pandas.read_csv to load the given training csv file.
        df = pd.read_csv(path)
        texts = []  # each element of texts is a single sentence.
        
        ### Your codes go here (10 points) ### ###COMPLETE####
        df.dropna(inplace=True) #remove NaN from dataframe
        for sentence in df['reviewText']:
            tokenized_sent = nltk.tokenize.sent_tokenize(sentence) #tokenize each review into sentences
            for sent in tokenized_sent: #for each sentence in tokenized_sent, append into texts
              texts.append(sent)

        print ('{} reviews loaded. {} sentences.'.format(df.shape[0], len(texts)))

        
        # Step 2: pass the list of all sentences from step 1 (texts) to WordIndexer.
        # Use its fit_transform function to turn list of sentences into list of lists of word indices in the sentences.
        # Keep the word ordering.
        print ('Indexing the corpus...')
        self.indexer = WordIndexer(min_word_occurrences=min_word_occurrences)
        corpus = self.indexer.fit_transform(texts)
        print ('Done indexing the corpus.')
        
        
        # Step 3: go through the results (corpus) from step 2 and gather (center, context) in comatrix,
        # which is a collections.Counter object.
        # In the Counter, keys are (center, context) pairs
        # values are the number of their co-occurrence as defined in the Glove paper.
        print ('Constructing the co-occurrence matrix...')
        ### Your codes go here (10 points) ###
        ####COMPLETE#####
        #use itertools to find combinations in corpus
        combinations = [tuple(itertools.combinations(word, 2)) for word in corpus]
        combinations = itertools.chain(*combinations)
        #use counter class to count number of combinations
        comatrix = Counter(combinations)
        print ("Done constructing the co-occurrence matrix.")


        # save the comatrix to file                    
        with open('./comatrix.pkl', 'wb') as out_f:
            pickle.dump(comatrix, out_f)

        # Step 4: flatten the co-occurrence matrix and store the center, context, and X_ij
        # in three lists: self.left (center word), self.right (context word), self.n_occurrences (X_ij)
        self.left, self.right, self.n_occurrences = [], [], [] #include self
        ### Your codes go here (10 points) ###
        ###COMPLETE#####
        comatrix_dict = dict(comatrix)
        keys = list(comatrix_dict.keys())
        for key in keys:
          x, y = key
          self.left.append(x)
          self.right.append(y)

        self.n_occurrences = list(comatrix_dict.values())
        print("Number of entries = ", len(comatrix)) #more than his (i have more word therefore will be greater), should remove non english words using regex soon. 
        
    def __getitem__(self, index):
        return self.left[index], self.right[index], self.n_occurrences[index]
    
    def __len__(self):
        return len(self.left)
    
def load_pretrained_wv(path):
    """
        Load the pretrained word vectors downloaded from Stanford NLP.
    """
    wv = {}
    with open(path, 'r',encoding='utf8') as f:
        for line in f.readlines():
            items = line.split(' ')
            wv[items[0]] = torch.DoubleTensor([float(a) for a in items[1:]])
    return wv

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chase\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chase\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Test of WordIndexer and AmazonReviewGloveDatabase

In [106]:
from torch import nn
import torch.functional as F

class GloveModel(nn.Module):
    def __init__(self, word_indexer, wv = None, word_dims = 300, BASE_STD = 0.01, random_state = 0):
        """ Specify and initialize the parameters of the Glove network.
        """
        super(GloveModel, self).__init__()
        num_words = word_indexer.n_words
        
        torch.manual_seed(random_state)
        
        # initialize left and right word vectors
        self.L_vecs = (torch.randn((num_words, word_dims))  * BASE_STD)
        self.R_vecs = (torch.randn((num_words, word_dims))  * BASE_STD)
       
        if wv is not None:
            num_replaced = 0
            for i in range(num_words):
                word = word_indexer.index_to_word[i]
                if word in wv:
                    num_replaced += 1
                    self.L_vecs[i] = wv[word]
                    self.R_vecs[i] = wv[word]
            print (f'Replaced {float(num_replaced) / num_words}')
            
        self.L_vecs.requires_grad_()
        self.R_vecs.requires_grad_()
        
        # gather the trainable parameters
        self.parameters = [self.L_vecs, self.R_vecs]
        
    def forward(self, left_indices, right_indices):
        """ Implement w_i^t w_j (the left-hand-side of Eq. (6) in the Glove paper)
        
            left_indices: torch.Tensor, a batch of center words
            right_indices: torch.Tensor, a batch of context words, of the same shape of left_indices.
            
            left_indices[i] and right_indices[i] is the i-th pair in the training data.
            
            return: torch.Tensor of the same shape of left_indices
        """
        ### Your codes go here (10 points) ###
        X = []
        for i in range(int(left_indices.size()[0])):
          X.append(self.L_vecs[left_indices[i]].dot(self.R_vecs[right_indices[i]]))
        
        X = torch.DoubleTensor(X)
        return X






## Model training, validating, and saving

### First define some constants




In [71]:
import torch
from tqdm import tqdm

# this will automatically place all tensor on GPU with type Double.
# if you are not running on GPU, change this line to
# torch.set_default_tensor_type('torch.DoubleTensor') GPU = torch.cuda.DoubleTensor
torch.set_default_tensor_type('torch.cuda.DoubleTensor')

# set up a couple of parameters and hyper-parameters

# number of epoches to train the model
NUM_EPOCH = 25
# size of mini-batches
BATCH_SIZE = 512

# dimension of word vectors. The integer should be the same as the dimension of
# pretrained word vectors.
NUM_DIMS = 300

# how to many words to the right to pair with the center word
WINDOW_SIZE = 10

# two hyper-parameters in Eq. (9) of the paper
x_max = 100
alpha = 0.75

# input file containing Amazon review texts.
train_path = './small.csv'

# where your model is saved.
save_path = './glove_model_{}.pt'

# optional word vectors pretrained
pretrained_wv = './glove.6B.{}d.txt'.format(NUM_DIMS)
print (pretrained_wv)

./glove.6B.300d.txt


In [72]:
# load pretrained word vectors
wv = load_pretrained_wv(pretrained_wv)

In [73]:
print (wv['good'])

tensor([-1.3602e-01, -1.1594e-01, -1.7078e-02, -2.9256e-01,  1.6149e-02,
         8.6472e-02,  1.5759e-03,  3.4395e-01,  2.1661e-01, -2.1366e+00,
         3.5278e-01, -2.3909e-01, -2.2174e-01,  3.6413e-01, -4.5021e-01,
         1.2104e-01, -1.5596e-01, -3.8906e-02, -2.9419e-03,  1.6009e-02,
        -1.1620e-01,  3.8680e-01,  3.5109e-01,  9.7426e-02, -1.2425e-02,
        -1.7864e-01, -2.3259e-01, -2.6960e-01,  4.1083e-02, -7.6194e-02,
        -2.3362e-01,  2.0919e-01, -2.7264e-01,  5.4967e-02, -1.8055e+00,
         5.6348e-01, -1.2778e-01,  2.3147e-01, -5.8820e-03, -2.6630e-01,
         4.1187e-01, -3.7162e-01, -2.0600e-01, -1.9619e-01, -4.3945e-03,
         1.2513e-01,  4.6638e-01,  4.5159e-01, -1.5000e-01,  5.9589e-03,
         5.9070e-02, -4.1440e-01,  6.1035e-02, -2.1117e-01, -4.0988e-01,
         5.6393e-01,  2.3021e-01,  2.7240e-01,  4.9364e-02,  1.4239e-01,
         4.1841e-01, -1.3983e-01,  3.4826e-01, -1.0745e-01, -2.5002e-01,
        -3.2554e-01,  3.3343e-01, -3.5617e-01,  2.0

### Then define the training, validation, and test data.
*   Use the AmazonReviewGloveDataset class to read train dataset.
*   Define DataLoader wrapping around the Dataset objects



In [65]:
# load text data and turn them into a DataLoader object.
train_dataset = AmazonReviewGloveDataset(train_path, right_window = WINDOW_SIZE)
train_iter = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True)

99946 reviews loaded. 493526 sentences.
Indexing the corpus...
Tokenizing sentences into words...
Done implementing tokenization and taking out stopwords.
Building a dictionary with occurrences of words...
Removing words that don't meet the minimum word occurrences...
Vocabulary in word_to_index and index_to_word.
Done indexing the corpus.
Constructing the co-occurrence matrix...
Done constructing the co-occurrence matrix.
Number of entries =  7133935


In [66]:
print(train_iter)
print(train_dataset.left[len(train_dataset.left)-20:len(train_dataset.left)-1])
print(train_dataset[1])

[350, 350, 244, 1520, 1520, 1520, 1520, 355, 355, 1539, 4526, 4526, 2351, 2351, 2351, 12055, 12055, 12055, 3072]
(1, 3, 8)


### Third, start training.

*   You're required to use GPU to train the network, since GPU are ubiquitous (colab or SandBox).
*   Complete the function train_and_validate.



In [107]:
import math
from torch.autograd import Variable

# decide whether to use cpu or gpu
device = torch.device('cuda')
# initialize the Glove model
model = GloveModel(train_dataset.indexer, wv, word_dims = NUM_DIMS)

# make sure you use weight_decay to activate the L2 regularization
optimizer = torch.optim.Adam(model.parameters, weight_decay=1e-8)

def train_and_validate(train_iter):
    best_loss = -1
    best_epoch = -1
    to_save = {}
    
    for epoch in range(NUM_EPOCH):
        model.train()
        epoch_loss = 0
        num_batches = len(train_iter)
        it = 0
        for l, r, n_lr in train_iter:
            it += 1
            optimizer.zero_grad()
            # Implement the loss function in Eq. (16) of the paper, in three steps.
            # Step 1. find the prediction of log(X_ij) using the model 
            ### Your codes go here (3 points) ###
            pred = model.forward(l,r)
            # Step 2. compute the weights f(X_ij). See Eq. (9) of the Glove paper.
            ### Your codes go here (3 points) ###
            weights = []
            i = 0;
            for x_ij in n_lr:
              pred[i] = pred[i] - math.log(x_ij,10)
              i += 1
              if x_ij < x_max: 
                weight = (x_ij/x_max)**alpha
              else:
                weight = 1
              weights.append(weight)
            weights = torch.DoubleTensor(weights)
            # Step 3. compute the loss in Eq. (16) using the predictions and the weights
            ### Your codes go here (4 points) ###
            loss = 0
            for i in range(int(pred.size()[0])):
              loss += weights[i] * (pred[i]** 2) #where weight = f(x_ij) 

            var_loss = Variable(loss.data, requires_grad=True)
            # tracking the averaged loss
            epoch_loss += loss.item()
            #print("Epoch Loss: ", epoch_loss, "Iteration: ", it)
            # gradient descent, don't change the following two lines
            var_loss.backward()
            optimizer.step()
        print(f'Training epoch = {epoch}, epoch loss = {epoch_loss / num_batches}')

        # record the model state_dict() for saving later
        #to_save = {
         #   'epoch': epoch,
          #  'model_state_dict': model.state_dict()
        #}
        #torch.save(to_save, save_path.format(epoch))
        #print (save_path.format(epoch))
    

Replaced 0.8538461538461538


In [108]:
train_and_validate(train_iter)

Training epoch = 0, epoch loss = 2008.167551130877
Training epoch = 1, epoch loss = 2008.167551130886
Training epoch = 2, epoch loss = 2008.167551130879


KeyboardInterrupt: 

## Retrieve similar words

In [ ]:
from sklearn.preprocessing import no4
rmalize

test_aspect_words = ['phone', 'case', 'battery', 'headset', 'charger', 'quality', 'screen', 'bluetooth', 'price', 'device']
test_sentimental_words = ['great', 'good', 'well', 'works', 'better', 'little', 'easy', 'nice', 'new', 'long']

glove = load_model(save_path.format(0), train_dataset.indexer)
avg_word_vectors = (glove.L_vecs.to('cpu') + glove.R_vecs.to('cpu')) / 2
avg_word_vectors = avg_word_vectors.detach().numpy()

n_words = train_dataset.indexer.n_words

row_normalized = normalize(avg_word_vectors)
sim = row_normalized.dot(row_normalized.T)

for w in test_aspect_words:
    w_idx = train_dataset.indexer.word_to_index[w]
    l = []
    for i in range(n_words):
        l.append((i, sim[w_idx, i]))
    l = sorted(l, key = lambda x:x[1], reverse = True)
    for i in range(10):
        print (f'{train_dataset.indexer.index_to_word[l[i][0]]}: {l[i][1]}')
        
for w in test_sentimental_words:
    w_idx = train_dataset.indexer.word_to_index[w]
    l = []
    for i in range(n_words):
        l.append((i, sim[w_idx, i]))
    l = sorted(l, key = lambda x:x[1], reverse = True)
    for i in range(10):
        print (f'{train_dataset.indexer.index_to_word[l[i][0]]}: {l[i][1]}')